# Project Setup
My initial conditions are guided by two key points:
    1. I don't want my computer to experience heat death
    2. I want to see, quickly, the springs acting as a retention force on the atoms
This is why my spring constant **k = 10** (rather than 1) and why my step **dt = 0.001**. I initially had my randomized velocity range from -1 to 1 but I kept getting the lattice exploding, so I made velocity's randomized range from **-0.5 to 0.5**. 

I'm still having problems with the system to where I can see a spring is acting but not with enough "stiffness" in the spring despite raising k to 10. However, it is clear that the system's springs are acting on the atoms. I think a way of doing this properly would to vectorize the position components in the force calculation sections of each integration method, but my attempts to do so didn't show any noticable change.

^^^I am keeping this in, as I just learned (a few days after I wrote that) that the behaviour described is due to the springs being undamped. Learning is fun!

References for forward/backward Euler are same as in slides shown in class and I did ask Ryan Hankee a brief question about python class/object structure and how to make a constructor

In [14]:
n = 3
k = 3 #2 or higher makes it faster to see springs act 
dt = 0.001 
v_rand = 1
class crystal:
    def __init__(self):
        self.atoms = []
        self.springs = [] 
        self.neighbors = {}
        self.KE = 0
        self.PE = 0 
        self.totalEnergy = 0 

        # making atoms (limited to cubic lattices due to the 1-dimensional input value)
        for z in range(-1, n-1):
            for y in range(-1, n-1):
                for x in range(-1, n-1):
                    atom = sphere(pos=vector(x, y, z), radius=0.2, color=color.blue)
                    atom.velocity = vector(random.uniform(-v_rand, v_rand), random.uniform(-v_rand, v_rand), random.uniform(-v_rand, v_rand))
                    atom.index = len(self.atoms)
                    atom.initPos = atom.pos
                    atom.mass = 1
                    self.atoms.append(atom)

        # Making axial and diagonal springs
        for i, atomA in enumerate(self.atoms):
            for j, atomB in enumerate(self.atoms):
                if i < j: 
                    dist = mag(atomA.pos - atomB.pos)
                    if dist == 1:  # axial
                        spring = helix(pos=atomA.pos, axis=atomB.pos - atomA.pos, radius=0.1, color=color.orange)
                        spring.atomA = atomA
                        spring.atomB = atomB
                        self.springs.append(spring)
                    elif dist == sqrt(2):  # diagonal
                        spring = helix(pos=atomA.pos, axis=atomB.pos - atomA.pos, radius=0.1, color=color.green)
                        spring.atomA = atomA
                        spring.atomB = atomB
                        self.springs.append(spring)
                        
                    if atomA not in self.neighbors:
                        self.neighbors[atomA] = []
                    if atomB not in self.neighbors:
                        self.neighbors[atomB] = []
                    self.neighbors[atomA].append(atomB)
                    self.neighbors[atomB].append(atomA)

        # getting rid of dupes
        for atom in self.neighbors:
            self.neighbors[atom] = set(self.neighbors[atom])

    def forwardsEuler(self, k, dt):
        self.KE = 0
        self.PE = 0
        
        for atomA in self.atoms:
            totalForce = vector(0,0,0)
            for atomB in self.neighbors[atomA]:
                forceA = -k*(atomA.pos-atomA.initPos)
                forceB = -k*(atomB.pos-atomB.initPos)
                forceAB = forceA+forceB
                totalForce += forceAB

            # calculate new motion vectors
            newVelocity = atomA.velocity + dt*totalForce/atomA.mass
            newPos = atomA.pos + dt*atomA.velocity
            atomA.velocity = newVelocity
            atomA.pos = newPos

            # calculate energy
            KE_atom = 0.5 * atomA.mass * mag2(newVelocity)
            PE_atom = 0.5 * k * mag2(atomA.pos - atomA.initPos)
            self.KE += KE_atom
            self.PE += PE_atom

        self.totalEnergy = self.KE + self.PE
        print(self.totalEnergy)
        self.update_springs()
    
    
    def backwardsEuler(self, k, dt):
        #Initializing to 0
        self.KE = 0
        self.PE = 0
        
        for atomA in self.atoms:
            total_force = vector(0,0,0)
            for atomB in self.neighbors[atomA]:
                force1 = -k*(atomA.pos-atomA.initPos)
                force2 = -k*(atomB.pos-atomB.initPos)
                force = force1+force2
                total_force += force
                
            # calculate new motion vectors
            acceleration = total_force/atomA.mass
            newVelocity = atomA.velocity + acceleration * dt
            newPos = atomA.pos + dt*newVelocity
            atomA.velocity = newVelocity
            atomA.pos = newPos

            # calculate energy
            KE_atom = 0.5 * atomA.mass * mag2(newVelocity)
            PE_atom = 0.5 * k * mag2(atomA.pos - atomA.initPos)
            self.KE += KE_atom
            self.PE += PE_atom

        self.totalEnergy = self.KE + self.PE
        print(self.totalEnergy)
        self.update_springs()
        
        
    def trapIntegration(self, k, dt):
        self.KE = 0
        self.PE = 0
        
        for atomA in self.atoms:
            totalForce = vector(0,0,0)
            for atomB in self.neighbors[atomA]:
                forceA = -k*(atomA.pos-atomA.initPos)
                forceB = -k*(atomB.pos-atomB.initPos)
                forceAB = forceA+forceB
                totalForce += forceAB
        
            # calculate midpoint motion vectors
            midpoint_accel = totalForce / atomA.mass
            midpoint_velocity = atomA.velocity + 0.5 * dt * midpoint_accel
        
            # calculate motion vectors
            accel = totalForce / atomA.mass
            newVelocity = atomA.velocity + 0.5 * dt * (midpoint_accel + accel)
            newPos = atomA.pos + 0.5 * dt * (atomA.velocity + midpoint_velocity)
            atomA.velocity = newVelocity
            atomA.pos = newPos
        
            # calculate energy
            KE_atom = 0.5 * atomA.mass * mag2(newVelocity)
            PE_atom = 0.5 * k * mag2(atomA.pos - atomA.initPos)
            self.KE += KE_atom
            self.PE += PE_atom

        self.totalEnergy = self.KE + self.PE
        print(self.totalEnergy)
        self.update_springs()

    def trapIntegrationDamped(self, k, dt, c):
        self.KE = 0
        self.PE = 0
        
        for atomA in self.atoms:
            totalForce = vector(0,0,0)
            for atomB in self.neighbors[atomA]:
                forceA = -k*(atomA.pos-atomA.initPos)
                forceB = -k*(atomB.pos-atomB.initPos)
                forceAB = forceA+forceB
                totalForce += forceAB
        
            # calculate midpoint motion vectors
            midpoint_accel = totalForce / atomA.mass
            midpoint_velocity = atomA.velocity + 0.5 * dt * midpoint_accel
        
            # calculate motion vectors
            accel = totalForce / atomA.mass
            newVelocity = atomA.velocity + 0.5 * dt * (midpoint_accel + accel)
            newPos = atomA.pos + 0.5 * dt * (atomA.velocity + midpoint_velocity)
            atomA.velocity = newVelocity * c
            atomA.pos = newPos
        
            # calculate energy
            KE_atom = 0.5 * atomA.mass * mag2(newVelocity)
            PE_atom = 0.5 * k * mag2(atomA.pos - atomA.initPos)
            self.KE += KE_atom
            self.PE += PE_atom

        self.totalEnergy = self.KE + self.PE
        print(self.totalEnergy)
        self.update_springs()

    def update_springs(self):
        for spring in self.springs:
            atomA = spring.atomA
            atomB = spring.atomB     
            
            spring.pos = atomA.pos
            spring.axis = atomB.pos - atomA.pos


# forwards euler w/ undamped springs


In [6]:
from vpython import*
import random 
scene = canvas()

c = crystal()

while True:
   rate(100)
   c.forwardsEuler(k,dt)

<IPython.core.display.Javascript object>

15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757

15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757

15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757

15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757

15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757365807
15.175282757

KeyboardInterrupt: 

# backwards euler w/ undamped springs


In [7]:
from vpython import*
import random 
scene = canvas()

c = crystal()

while True:
   rate(100)
   c.backwardsEuler(k,dt)

<IPython.core.display.Javascript object>

11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975

11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975

11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975

11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975

...

11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975

11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975

11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975

11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975779765
11.898967975

KeyboardInterrupt: 

# trapezoidal integration w/ undamped springs

reference for equations used: https://en.wikipedia.org/wiki/Trapezoidal_rule

In [11]:
from vpython import*
import random 
scene = canvas()

c = crystal()

while True:
   rate(100)
   c.trapIntegration(k,dt)

<IPython.core.display.Javascript object>

3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212

3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212

3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212

3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212

3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212167017
3.3071180212

KeyboardInterrupt: 

# trapezoidal integration w/ damped springs

Damping is pretty easy to implement with my current class-object setup, I just add a damping coefficient to the newVelocity calculation and it will functionally mimic a damped spring. To see this a bit more clearly, I turned the initial velocity back up to range randomly from -1 to 1 and decreased the spring constant from 10 to 3. Through the total energy printout you can see the system losing energy.

reference for equations: https://restackor.com/physics/response/spring-mass-damper#:~:text=with%20shaft%20velocity-,Spring%2Dmass%2Ddamper%20theory%20applies%20the%20basic%20physics%20of%20F,forces%20acting%20on%20the%20suspension.

In [16]:
from vpython import*
import random 
scene = canvas()

c = crystal()
damp = 0.999
while True:
   rate(100)
   c.trapIntegrationDamped(k,dt, damp)

<IPython.core.display.Javascript object>

14.052954097314075
14.024862242073542
13.99682654245164
13.968846886193273
13.940923161267776
13.9130552558684
13.885243058411922
13.857486457538156
13.829785342109536
13.80213960121066
13.774549124147839
13.74701380044867
13.719533519861567
13.692108172355367
13.664737648118829
13.637421837560238
13.610160631306957
13.582953920204973
13.555801595318481
13.528703547929444
13.50165966953713
13.474669851857723
13.447733986823863
13.420851966584202
13.394023683502999
13.36724903015968
13.340527899348391
13.313860184077589
13.28724577756962
13.26068457326026
13.23417646479831
13.207721346045178
13.181319111074433
13.154969654171401
13.128672869832709
13.102428652765914
13.076236897889036
13.050097500330153
13.024010355426993
12.997975358726496
12.971992405984402
12.946061393164836
12.920182216439901
12.894354772189239
12.868578956999631
12.84285466766459
12.817181801183926
12.791560254763361
12.76598992581409
12.740470711952385
12.715002510999192
12.689585220979707
12.664218740122967
12.63

5.642777029727261
5.631497118444836
5.620239755705065
5.609004896433412
5.59779249564544
5.586602508446644
5.575434890032259
5.564289595687085
5.553166580785307
5.542065800790317
5.530987211254537
5.519930767819238
5.508896426214368
5.497884142258365
5.486893871857992
5.475925571008148
5.4649791957917015
5.454054702379315
5.443152047029258
5.432271186087246
5.421412075986257
5.41057467324636
5.3997589344745425
5.388964816364528
5.378192275696614
5.367441269337497
5.3567117542400915
5.346003687443366
5.335317026072166
5.3246517273370495
5.3140077485341
5.303385047044781
5.292783580335739
5.282203305958648
5.2716441815500374
5.2611061648311175
5.25058921360762
5.2400932857696185
5.229618339291366
5.219164332231122
5.208731222730993
5.198318969016753
5.1879275293976885
5.177556862266422
5.167206926098752
5.156877679453481
5.146569080972253
5.1362810893793895
5.1260136634817215
5.115766762168421
5.105540344410845
5.0953343692623685
5.0851487958582124
5.074983583415291
5.064838691232044
5.0

2.279424555701086
2.27486798601424
2.270320524910198
2.265782154180902
2.2612528556546936
2.2567326111962407
2.2522214027064598
2.2477192121224494
2.243226021417416
2.238741812600603
2.2342665677172144
2.2298002688483476
2.22534289811092
2.2208944376575963
2.2164548696767183
2.212024176392235
2.2076023400636267
2.20318934298584
2.198785167489211
2.1943897959393994
2.190003210737317
2.185625394319053
2.181256329155809
2.1768959977538267
2.172544382654317
2.1682014664333913
2.16386723170199
2.159541661105818
2.1552247373252675
2.1509164430753547
2.1466167611056473
2.1423256742001966
2.1380431651774705
2.1337692168902804
2.129503812225717
2.125246934105078
2.1209985654838013
2.1167586893514
2.1125272887313864
2.108304346681212
2.1040898462921964
2.0998837706894586
2.09568610303185
2.09149682651189
2.0873159243556922
2.0831433798229053
2.0789791762066394
2.0748232968334017
2.070675725063032
2.066536444288631
2.0624054379364987
2.058282689466063
2.0541681823698204
2.050061900173263
2.045963

0.9375161703046109
0.9356420754801721
0.9337717269712871
0.9319051172890713
0.9300422389596106
0.9281830845239303
0.9263276465379672
0.9244759175725377
0.9226278902133104
0.9207835570607739
0.9189429107302093
0.9171059438516596
0.9152726490699
0.9134430190444092
0.9116170464493396
0.9097947239734874
0.9079760443202644
0.9061610002076682
0.9043495843682529
0.902541789549101
0.9007376085117924
0.8989370340323771
0.8971400589013464
0.8953466759236027
0.8935568779184315
0.8917706577194726
0.8899880081746911
0.88820892214635
0.8864333925109796
0.88466141215935
0.8828929739964436
0.8811280709414245
0.8793666959276125
0.8776088419024531
0.8758545018274901
0.874103668678337
0.8723563354446492
0.8706124951300953
0.8688721407523303
0.8671352653429663
0.8654018619475458
0.8636719236255125
0.8619454434501853
0.8602224145087283
0.8585028299021251
0.8567866827451507
0.8550739661663435
0.8533646733079768
0.8516587973260342
0.8499563313901793
0.8482572686837306
0.8465616024036317
0.8448693257604268
0.

...

0.0012942622094607847
0.0012916749793040728
0.0012890929210204439
0.001286516024271324
0.0012839442787388059
0.0012813776741256067
0.0012788162001550298
0.00127625984657092
0.0012737086031376248
0.0012711624596399526
0.0012686214058831326
0.0012660854316927717
0.0012635545269148183
0.0012610286814155154
0.0012585078850813657
0.0012559921278190883
0.0012534813995555777
0.0012509756902378663
0.0012484749898330802
0.001245979288328404
0.0012434885757310358
0.0012410028420681493
0.001238522077386855
0.0012360462717541589
0.0012335754152569222
0.0012311094980018236
0.0012286485101153178
0.0012261924417435975
0.0012237412830525516
0.0012212950242277301
0.0012188536554742985
0.0012164171670170057
0.0012139855491001385
0.0012115587919874873
0.0012091368859623044
0.0012067198213272655
0.0012043075884044321
0.0012019001775352121
0.0011994975790803194
0.0011970997834197376
0.0011947067809526815
0.0011923185620975568
0.0011899351172919243
0.0011875564369924578
0.0011851825116749098
0.0011828133318

0.0005906973734661845
0.0005895165694166257
0.0005883381257943619
0.000587162037880899
0.0005859883009671751
0.0005848169103535416
0.0005836478613497447
0.0005824811492749066
0.0005813167694575062
0.0005801547172353606
0.0005789949879556072
0.0005778375769746837
0.0005766824796583116
0.0005755296913814746
0.000574379207528403
0.0005732310234925536
0.0005720851346765921
0.0005709415364923735
0.0005698002243609253
0.0005686611937124278
0.0005675244399861967
0.0005663899586306643
0.0005652577451033616
0.0005641277948708999
0.0005630001034089531
0.0005618746662022386
0.0005607514787445002
0.0005596305365384899
0.0005585118350959495
0.0005573953699375928
0.0005562811365930875
0.000555169130601038
0.0005540593475089663
0.0005529517828732959
0.0005518464322593322
0.0005507432912412459
0.0005496423554020546
0.0005485436203336058
0.000547447081636559
0.0005463527349203676
0.0005452605758032617
0.000544170599912231
0.0005430828028830065
0.0005419971803600434
0.0005409137279965035
0.0005398324414

0.00027230331389841746
0.00027175897957393454
0.0002712157333737663
0.0002706735731227521
0.0002701324966500797
0.0002695925017892762
0.00026905358637819936
0.0002685157482590294
0.0002679789852782596
0.00026744329528668835
0.00026690867613941025
0.00026637512569580756
0.00026584264181954165
0.00026531122237854435
0.0002647808652450097
0.00026425156829538494
0.0002637233294103624
0.0002631961464748711
0.0002626700173780678
0.0002621449400133291
0.00026162091227824243
0.0002610979320745981
0.00026057599730838115
0.00026005510588976167
0.000259535255733088
0.00025901644475687753
0.00025849867088380854
0.0002579819320407118
0.0002574662261585624
0.0002569515511724715
0.0002564379050216777
0.00025592528564953934
0.00025541369100352595
0.0002549031190352099
0.0002543935677002585
0.00025388503495842567
0.0002533775187735438
0.0002528710171135155
0.00025236552795030554
0.0002518610492599329
0.00025135757902246226
0.0002508551152219964
0.00025035365584666764
0.0002498531988886302
0.00024935374

0.0001237820436813448
0.0001235346033760258
0.0001232876577038771
0.00012304120567612704
0.0001227952463059805
0.00012254977860861482
0.00012230480160117622
0.00012206031430277545
0.00012181631573448421
0.00012157280491933096
0.00012132978088229724
0.00012108724265031351
0.00012084518925225552
0.00012060361971894025
0.0001203625330831221
0.00012012192837948892
0.00011988180464465834
0.00011964216091717367
0.00011940299623750024
0.00011916430964802148
0.00011892610019303507
0.0001186883669187492
0.00011845110887327865
0.00011821432510664094
0.00011797801467075275
0.00011774217661942592
0.0001175068100083637
0.00011727191389515698
0.00011703748733928053
0.00011680352940208935
0.00011657003914681459
0.00011633701563856008
0.00011610445794429858
0.00011587236513286793
0.00011564073627496736
0.00011540957044315368
0.00011517886671183781
0.00011494862415728083
0.00011471884185759042
0.00011448951889271709
0.00011426065434445055
0.000114032247296416
0.00011380429683407048
0.000113576802044699

KeyboardInterrupt: 